# IMPORTACIÓN LIBRERIAS


In [2]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import re

import warnings
warnings.filterwarnings('ignore')

# CARGA DE ARCHIVOS

In [3]:
def load_dataset(file_path, file_type=None, separator=None, encoding='utf-8', **kwargs):
    """
    Loads a dataset in different formats, with support for custom separators, encoding, and more options.
    """
    # If the file type is not specified, infer from file extension
    if not file_type:
        file_type = file_path.split('.')[-1].lower()

    # Load according to the file type
    if file_type == 'csv':
        return pd.read_csv(file_path, sep=separator or ',', encoding=encoding, **kwargs)
    elif file_type in ['xls', 'xlsx']:
        return pd.read_excel(file_path, **kwargs)
    elif file_type == 'json':
        return pd.read_json(file_path, encoding=encoding, **kwargs)
    else:
        raise ValueError(f"File format '{file_type}' not supported. Use 'csv', 'excel', or 'json'.")

### - ARCHIVO CLIENTES ACTIVOS HASTA 15/9/25

In [9]:
df = load_dataset('../data/Llistat Abonats actius 15.09.2025.xlsx')
df = df[1:].reset_index(drop=True)

In [10]:
df

,IdPersona,Sexo,Edad,FNacimiento,DireccionCompleta,CodigoPostal,FechaAlta,FAntiguedad,FechaBaja,MotivoBaja,TipoAbono
0,334,Hombre,82,04/12/1942,"CARRER CERAMICA, 37 BXS",08035,22/03/2011,29/11/1955,,,AVET - ABONATS VETERANS
1,452,Hombre,85,01/08/1940,"CARRER HEDILLA, 410 4-2",08031,18/09/1956,18/09/1956,,,AVET - ABONATS VETERANS
2,642,Hombre,77,02/02/1948,"CARRER SANT TOMAS, 71-73 ENTLO-2",08032,22/03/2011,05/03/1958,,,AVET - ABONATS VETERANS
3,849,Hombre,85,10/04/1940,"CALLE MARI CURI, 40",08042,04/08/1958,04/08/1958,,,CL04 - JUBILATS SENSE DRET A US
4,891,Hombre,78,23/09/1946,"CARRER ROSSELLO, 31 AT-1",08029,22/03/2011,07/01/1959,,,AVET - ABONATS VETERANS
...,...,...,...,...,...,...,...,...,...,...,...
5863,117580,Mujer,21,26/06/2004,"CARRER TORRENT DE CAN MARINER, 33, 3-3",08031,15/09/2025,15/09/2025,,,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS)
5864,117582,Mujer,24,30/10/2000,"PASSEIG FABRA I PUIG, 391",08031,15/09/2025,15/09/2025,01/12/2025,Canvi Domicili,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS)
5865,117584,Mujer,52,25/07/1973,"VIA JULIA, 124 5º3ª",08016,15/09/2025,15/09/2025,01/10/2025,,AA00 - ADULTS ( 26 A 64 ANYS )
5866,117585,Hombre,20,09/11/2004,"CARRER MOSEN JACIN VERDAGUER, 108",NaN,15/09/2025,15/09/2025,,,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS)


#### GESTIÓN DE COLUMNAS Y FILAS

In [11]:
def preparar_datos_iniciales(df: pd.DataFrame, columnas_a_eliminar: list,   columnas_a_renombrar: dict,
    columnas_numericas: list, columnas_fechas: list) -> pd.DataFrame:
    """
    Prepara un DataFrame para análisis exploratorio de datos (EDA).
    
    Parámetros:
        df (pd.DataFrame): DataFrame original
        columnas_a_eliminar (list): Columnas que se eliminarán del DataFrame
        columnas_a_renombrar (dict): Diccionario con columnas a renombrar {original: nuevo_nombre}
        columnas_numericas (list): Columnas que deben convertirse a tipo numérico
        columnas_fechas (list): Columnas que deben convertirse a tipo datetime
    
    Retorna:
        pd.DataFrame: DataFrame transformado
    """
    df_eda = df.copy()

    # Eliminar columnas
    df_eda.drop(columns=columnas_a_eliminar, inplace=True, errors='ignore')

    # Renombrar columnas
    df_eda.rename(columns=columnas_a_renombrar, inplace=True)

    # Conversión de columnas numéricas
    for col in columnas_numericas:
        if col in df_eda.columns:
            df_eda[col] = pd.to_numeric(df_eda[col], errors='coerce')

    # Conversión de columnas de fecha
    for col in columnas_fechas:
        if col in df_eda.columns:
            df_eda[col] = pd.to_datetime(df_eda[col], errors='coerce', dayfirst=True)

    return df_eda

In [12]:
columnas_a_eliminar = ['FechaBaja', 'MotivoBaja', 'DireccionCompleta', 'CodigoPostal']
columnas_a_renombrar = {'FechaAlta': 'FechaInscripcion', 'TipoAbono': 'TipoAbonoActual'}
columnas_numericas = ['Edad']
columnas_fechas = ['FNacimiento', 'FechaInscripcion', 'FAntiguedad']

df_eda = preparar_datos_iniciales(df, columnas_a_eliminar, columnas_a_renombrar,
    columnas_numericas,   columnas_fechas)

In [13]:
df_eda

,IdPersona,Sexo,Edad,FNacimiento,FechaInscripcion,FAntiguedad,TipoAbonoActual
0,334,Hombre,82,1942-12-04,2011-03-22,1955-11-29,AVET - ABONATS VETERANS
1,452,Hombre,85,1940-08-01,1956-09-18,1956-09-18,AVET - ABONATS VETERANS
2,642,Hombre,77,1948-02-02,2011-03-22,1958-03-05,AVET - ABONATS VETERANS
3,849,Hombre,85,1940-04-10,1958-08-04,1958-08-04,CL04 - JUBILATS SENSE DRET A US
4,891,Hombre,78,1946-09-23,2011-03-22,1959-01-07,AVET - ABONATS VETERANS
...,...,...,...,...,...,...,...
5863,117580,Mujer,21,2004-06-26,2025-09-15,2025-09-15,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS)
5864,117582,Mujer,24,2000-10-30,2025-09-15,2025-09-15,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS)
5865,117584,Mujer,52,1973-07-25,2025-09-15,2025-09-15,AA00 - ADULTS ( 26 A 64 ANYS )
5866,117585,Hombre,20,2004-11-09,2025-09-15,2025-09-15,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS)


In [14]:
def eda_basica(df: pd.DataFrame, nombre_df: str = "DataFrame") -> None:
    """
    Realiza un análisis exploratorio básico sobre un DataFrame:
    - Identifica variables numéricas y categóricas
    - Detecta valores nulos y muestra una visualización si los hay
    - Revisa duplicados (filas y columnas)

    Parámetros:
        df (pd.DataFrame): El DataFrame a analizar
        nombre_df (str): Nombre para mostrar del DataFrame (opcional)
    """
    print(f"\n📋 Análisis EDA básico de: {nombre_df}")

    # 1. Tipos de variables
    print("\n📌 Tipos de Variables:")
    num_vbles = df.select_dtypes(include='number').columns.tolist()
    cat_vbles = df.select_dtypes(exclude='number').columns.tolist()
    print(f"🔢 Variables Numéricas: {num_vbles}")
    print(f"🔠 Variables Categóricas: {cat_vbles}")

    # 2. Valores nulos
    print("\n🕳️ Variables con valores nulos:")
    missing = df.isnull().sum()
    missing = missing[missing > 0].sort_values(ascending=False)
    missing_percentage = (missing / len(df)) * 100
    missing_df = pd.DataFrame({
        'Total Missing': missing,
        'Percentage Missing': missing_percentage
    })

    if not missing.empty:
        display(missing_df)
        plt.figure(figsize=(10, 6))
        missing.plot(kind='barh', color='salmon')
        plt.title("Variables con Valores Nulos")
        plt.xlabel("Cantidad de valores nulos")
        plt.gca().invert_yaxis()
        plt.grid(True, axis='x', linestyle='--', alpha=0.7)
        plt.show()
    else:
        print("✅ No hay valores nulos en el dataset.")

    # 3. Filas duplicadas
    print("\n📎 Filas duplicadas:")
    duplicadas = df.duplicated().sum()
    if duplicadas > 0:
        print(f"🔴 Hay {duplicadas} filas duplicadas.")
        display(df[df.duplicated()])
    else:
        print("✅ No hay filas duplicadas.")

    # 4. Columnas duplicadas
    print("\n📎 Columnas duplicadas:")
    columnas_duplicadas = df.T.duplicated().sum()
    if columnas_duplicadas > 0:
        print(f"🔴 Hay {columnas_duplicadas} columnas duplicadas.")
    else:
        print("✅ No hay columnas duplicadas.")

In [15]:
eda_basica(df_eda, nombre_df="Clientes Activos")


📋 Análisis EDA básico de: Clientes Activos

📌 Tipos de Variables:
🔢 Variables Numéricas: ['IdPersona', 'Edad']
🔠 Variables Categóricas: ['Sexo', 'FNacimiento', 'FechaInscripcion', 'FAntiguedad', 'TipoAbonoActual']

🕳️ Variables con valores nulos:
✅ No hay valores nulos en el dataset.

📎 Filas duplicadas:
✅ No hay filas duplicadas.

📎 Columnas duplicadas:
✅ No hay columnas duplicadas.


In [16]:
def filtrar_por_fecha(df: pd.DataFrame, columna_fecha: str, fecha_limite: str) -> pd.DataFrame:
    """
    Filtra un DataFrame manteniendo solo las filas donde la fecha en la columna especificada
    es menor o igual a una fecha límite.

    Parámetros:
        df (pd.DataFrame): DataFrame a filtrar
        columna_fecha (str): Nombre de la columna de tipo fecha
        fecha_limite (str): Fecha límite en formato 'YYYY-MM-DD'

    Retorna:
        pd.DataFrame: DataFrame filtrado
    """
    if columna_fecha not in df.columns:
        raise ValueError(f"La columna '{columna_fecha}' no existe en el DataFrame.")

    # Convertir la fecha límite a tipo datetime
    fecha_limite = pd.to_datetime(fecha_limite, errors='coerce')

    if fecha_limite is pd.NaT:
        raise ValueError("La fecha límite no es válida. Usa el formato 'YYYY-MM-DD'.")

    # Asegurar que la columna sea de tipo datetime
    df_filtrado = df.copy()
    df_filtrado[columna_fecha] = pd.to_datetime(df_filtrado[columna_fecha], errors='coerce')

    # Aplicar el filtro
    df_filtrado = df_filtrado[df_filtrado[columna_fecha] <= fecha_limite]

    return df_filtrado


In [17]:
# Filtrar por 'FAntiguedad'
df_features_1sep25 = filtrar_por_fecha(df_eda, 'FAntiguedad', '2025-09-01')

# También podrías filtrar por 'FechaInscripcion' si quisieras
df_features_1sep25 = filtrar_por_fecha(df_eda, 'FechaInscripcion', '2025-09-01')
df_features_1sep25

,IdPersona,Sexo,Edad,FNacimiento,FechaInscripcion,FAntiguedad,TipoAbonoActual
0,334,Hombre,82,1942-12-04,2011-03-22,1955-11-29,AVET - ABONATS VETERANS
1,452,Hombre,85,1940-08-01,1956-09-18,1956-09-18,AVET - ABONATS VETERANS
2,642,Hombre,77,1948-02-02,2011-03-22,1958-03-05,AVET - ABONATS VETERANS
3,849,Hombre,85,1940-04-10,1958-08-04,1958-08-04,CL04 - JUBILATS SENSE DRET A US
4,891,Hombre,78,1946-09-23,2011-03-22,1959-01-07,AVET - ABONATS VETERANS
...,...,...,...,...,...,...,...
5726,117425,Hombre,30,1995-02-11,2025-09-01,2025-09-01,AA00 - ADULTS ( 26 A 64 ANYS )
5727,117426,Mujer,46,1979-08-05,2025-09-01,2025-09-01,AA00 - ADULTS ( 26 A 64 ANYS )
5728,117427,Hombre,20,2004-11-19,2025-09-01,2025-09-01,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS)
5827,117537,Hombre,16,2008-11-18,2025-09-01,2025-09-01,AM00 - INFANTIL (DE 6 A 17 ANYS)


## FEATURE ENGINEERING

In [18]:
def excluir_valores(df: pd.DataFrame, columna: str, valores_a_excluir: list) -> pd.DataFrame:
    """
    Filtra un DataFrame excluyendo las filas que contienen ciertos valores en una columna específica.

    Parámetros:
        df (pd.DataFrame): DataFrame original
        columna (str): Nombre de la columna en la que se aplicará el filtro
        valores_a_excluir (list): Lista de valores que se quieren excluir

    Retorna:
        pd.DataFrame: DataFrame filtrado, sin los valores excluidos y con índice reseteado
    """
    if columna not in df.columns:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame.")

    df_filtrado = df[~df[columna].isin(valores_a_excluir)].reset_index(drop=True)
    return df_filtrado

In [19]:
tipos_a_excluir = ["EMP0 - EMPLEADOS CLUB SIN CUOTA",  "EMP1 - EMPLEATS D'ALTRES EMPRESES",  "CL02 - SOCIS NUMERARIS",    "CL01 - SOCIS D'HONOR"]

df_activos_filtrado = excluir_valores(df_features_1sep25, 'TipoAbonoActual', tipos_a_excluir)

In [20]:
def codificar_one_hot(df: pd.DataFrame, columna: str, drop_first: bool = True) -> pd.DataFrame:
    """
    Aplica codificación one-hot a una columna categórica de un DataFrame y elimina la columna original.

    Parámetros:
        df (pd.DataFrame): DataFrame original
        columna (str): Nombre de la columna categórica a codificar
        drop_first (bool): Si se elimina la primera categoría para evitar multicolinealidad (default=True)

    Retorna:
        pd.DataFrame: DataFrame con columnas one-hot y sin la columna original
    """
    if columna not in df.columns:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame.")

    # Aplicar codificación one-hot
    dummies = pd.get_dummies(df[columna], prefix=columna, drop_first=drop_first)

    # Concatenar y eliminar columna original
    df_codificado = pd.concat([df.drop(columns=[columna]), dummies], axis=1)

    return df_codificado

In [21]:
df_features_codificado = codificar_one_hot(df_activos_filtrado, 'Sexo')

In [22]:
df_features_codificado

,IdPersona,Edad,FNacimiento,FechaInscripcion,FAntiguedad,TipoAbonoActual,Sexo_Mujer
0,334,82,1942-12-04,2011-03-22,1955-11-29,AVET - ABONATS VETERANS,False
1,452,85,1940-08-01,1956-09-18,1956-09-18,AVET - ABONATS VETERANS,False
2,642,77,1948-02-02,2011-03-22,1958-03-05,AVET - ABONATS VETERANS,False
3,849,85,1940-04-10,1958-08-04,1958-08-04,CL04 - JUBILATS SENSE DRET A US,False
4,891,78,1946-09-23,2011-03-22,1959-01-07,AVET - ABONATS VETERANS,False
...,...,...,...,...,...,...,...
5483,117425,30,1995-02-11,2025-09-01,2025-09-01,AA00 - ADULTS ( 26 A 64 ANYS ),False
5484,117426,46,1979-08-05,2025-09-01,2025-09-01,AA00 - ADULTS ( 26 A 64 ANYS ),True
5485,117427,20,2004-11-19,2025-09-01,2025-09-01,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS),False
5486,117537,16,2008-11-18,2025-09-01,2025-09-01,AM00 - INFANTIL (DE 6 A 17 ANYS),False


In [23]:
def cambios_nombre_abonos(df: pd.DataFrame, columna: str, mapeo_manual: dict = None) -> pd.DataFrame:
    """
    Extrae un código de una columna de texto usando una expresión regular y un mapeo manual.

    Parámetros:
        df (pd.DataFrame): DataFrame de entrada
        columna (str): Nombre de la columna a procesar
        mapeo_manual (dict): Diccionario de mapeo manual para valores sin patrón

    Retorna:
        pd.DataFrame: Una copia del DataFrame con la columna transformada
    """

    if columna not in df.columns:
        raise ValueError(f"La columna '{columna}' no existe en el DataFrame.")

    # Expresión regular para códigos con formato "XXXX - ..."
    patron_codigo = re.compile(r"^([A-Z0-9]{2,5})\s*-\s*")

    # Usar un dict vacío si no se pasa uno manual
    mapeo_manual = mapeo_manual or {}

    def procesar_valor(valor):
        if pd.isna(valor):
            return None
        match = patron_codigo.match(str(valor).strip())
        if match:
            return match.group(1)
        return mapeo_manual.get(valor, valor)

    df_copia = df.copy()
    df_copia[columna] = df_copia[columna].apply(procesar_valor)
    return df_copia


| AbonoOriginal                                               | CodigoAbono |
|-------------------------------------------------------------|-------------|
| AA00 - ADULTS ( 26 A 64 ANYS )                              | AA00        |
| AR00 - TARJA ROSA (TARJETA GRATUÏTA)                        | AR00        |
| JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS)               | JO00        |
| AG00 - GENT GRAN (MES DE 65 ANYS)                           | AG00        |
| AT01 - ATUR TOTAL                                           | AT01        |
| AF00 - ADULTS C.S. (DIVENDRES TARDA MES CAP DE...           | AF00        |
| MA00 - ADULTS MATINS ( DE 7 A 16 MES CAPS DE S...           | MA00        |
| CR01 - CARNET ROSA (TARJETA REDUÏDA)                        | CR01        |
| FAMILIAR (PARES MES ELS MENORS DE 18 ANYS)                  | FA00        |
| QUOTA MANTENIMENT - MENSUAL                                 | QM01        |
| AG03 - GENT GRAN - TRIMESTRAL                               | AG03        |
| AM00 - INFANTIL (DE 6 A 17 ANYS)                             | AM00        |
| ATURATS MATI                                                | AT00        |
| AT00 - ATUR MATI                                            | AT00        |
| AC00 - COMERCIANTS (DE 7 A 9 I DE 13 A 17 MES ...            | AC00        |
| TEMP                                                        | TMP         |
| AA03 - ADULTS - TRIMESTRAL                                  | AA03        |
| MA03 - ADULTS MATINS - TRIMESTRAL                           | MA03        |
| AA12 - ADULTS - ANUAL                                       | AA12        |
| ATURATS TOTAL                                               | AT01        |
| FAMILIAR MONOPARENTAL (AMB CARNET MONOPARENTAL)             | FM01        |
| AR03 - TARJA ROSA - TRIMESTRAL                              | AR03        |
| MA06 - ADULTS MATINS - SEMESTRAL                            | MA06        |
| AVET - ABONATS VETERANS                                     | AVET        |
| AC03 - COMERCIANTS - TRIMESTRAL                             | AC03        |
| JO03 - QUOTA TRIMESTRAL JOVE                                | JO03        |
| FAMILIAR ANUAL                                              | FA12        |
| AP03 - PREINFANTIL3-5                                       | AP03        |
| AP00 - PREINFANTIL0-2                                       | AP00        |
| AF03 - ADULTS C.S - TRIMESTRAL                              | AF03        |
| AG12 - GENT GRAN - ANUAL                                    | AG12        |
| AM03 - INFANTIL-TRIMESTRAL                                  | AM03        |
| EMPF - FAMILIAR EMPLEADO                                    | EMPF        |
| CR03 - CARNET ROSA - TRIMESTRAL                             | CR03        |
| NI00 - ADULTS NITS                                          | NI00        |
| VIP                                                         | VIP         |
| CL04 - JUBILATS SENSE DRET A US                             | CL04        |
| QUOTA MANTENIMENT - TRIM.                                   | QM03        |
| MA12 - ADULTS MATINS ANUALS                                 | MA12        |
| AR12 - TARJA ROSA - ANUAL                                   | AR12        |
| APG03-GROUPON-SEMESTRAL                                     | APG03        |
| APG04-GROUPON-ANUAL                                     | APG04        |
| T07 - QUOTA I-10                                     | T07        |
| T12 - QUOTA TR-25                                     | T12        |
| T14 - QUOTA TR-15                                     | T14        |
| T16 - QUOTA                                     | T16        |
| T15 - QUOTA TR-10                                     | T15        |



In [24]:
mapeo_manual_abonos = {
    'FAMILIAR (PARES MES ELS MENORS DE 18 ANYS)': 'FA00',
    'QUOTA MANTENIMENT - MENSUAL': 'QM01',
    'FAMILIAR MONOPARENTAL (AMB CARNET MONOPARENTAL)': 'FM01',
    'TEMP': 'TEMP',
    'VIP': 'VIP',
    'FAMILIAR ANUAL': 'FA12',
    'QUOTA MANTENIMENT - TRIM.': 'QM03',
    'ATURATS TOTAL': 'AT01',
    'ATURATS MATI': 'AT00',
    "AA0 - PROMO 9'90€ (PRIMER MES)": 'AA0',
    "APG03-GROUPON-SEMESTRAL": 'APG03',
    "APG04-GROUPON-ANUAL": 'APG04',
    "T07 - QUOTA I-10": 'T07',
    "T12 - QUOTA TR-25": 'T12',
    "T14 - QUOTA TR-15": 'T14',
    "T16 - QUOTA": 'T16',
    "T15 - QUOTA TR-10": 'T15',
    'T08 - QUOTA TR-30': 'T08',
}

df_abonos_limpios = cambios_nombre_abonos( df_features_codificado,  columna='TipoAbonoActual',   mapeo_manual=mapeo_manual_abonos)


In [25]:
df_abonos_limpios

,IdPersona,Edad,FNacimiento,FechaInscripcion,FAntiguedad,TipoAbonoActual,Sexo_Mujer
0,334,82,1942-12-04,2011-03-22,1955-11-29,AVET,False
1,452,85,1940-08-01,1956-09-18,1956-09-18,AVET,False
2,642,77,1948-02-02,2011-03-22,1958-03-05,AVET,False
3,849,85,1940-04-10,1958-08-04,1958-08-04,CL04,False
4,891,78,1946-09-23,2011-03-22,1959-01-07,AVET,False
...,...,...,...,...,...,...,...
5483,117425,30,1995-02-11,2025-09-01,2025-09-01,AA00,False
5484,117426,46,1979-08-05,2025-09-01,2025-09-01,AA00,True
5485,117427,20,2004-11-19,2025-09-01,2025-09-01,JO00,False
5486,117537,16,2008-11-18,2025-09-01,2025-09-01,AM00,False


In [26]:
def crear_features_activos(df: pd.DataFrame,   fecha_corte_inicio: str,  fecha_corte: str,
                              columnas_fechas: dict = None,   id_col: str = 'IdPersona') -> pd.DataFrame:
    """
    Crea variables temporales de interés para análisis longitudinal de altas y abonos.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame con los datos originales.
    fecha_corte_inicio : str
        Fecha inicio del periodo de estudio (formato 'YYYY-MM-DD').
    fecha_corte : str
        Fecha de corte final (formato 'YYYY-MM-DD').
    columnas_fechas : dict, opcional
        Diccionario con nombres de columnas de fechas, con claves:
        - 'antiguedad' (por defecto 'FAntiguedad')
        - 'inscripcion' (por defecto 'FechaInscripcion')
    id_col : str, opcional
        Nombre de la columna que identifica a la persona (por defecto 'IdPersona').

    Retorna:
    --------
    pd.DataFrame
        DataFrame con las nuevas variables creadas y columnas originales importantes.
    """

    # Definir columnas fechas por defecto si no se pasan
    if columnas_fechas is None:
        columnas_fechas = {'antiguedad': 'FAntiguedad', 'inscripcion': 'FechaInscripcion'}

    # Copiar df para no modificar original
    df = df.copy()

    # Convertir columnas de fechas a datetime (por si acaso)
    df[columnas_fechas['antiguedad']] = pd.to_datetime(df[columnas_fechas['antiguedad']], errors='coerce')
    df[columnas_fechas['inscripcion']] = pd.to_datetime(df[columnas_fechas['inscripcion']], errors='coerce')

    # Convertir fechas de corte a Timestamp
    fecha_corte_inicio = pd.Timestamp(fecha_corte_inicio)
    fecha_corte = pd.Timestamp(fecha_corte)

    # 1. Antigüedad mínima por persona
    antiguedad_min = df.groupby(id_col)[columnas_fechas['antiguedad']].min().reset_index()

    # 2. Última inscripción por persona (más reciente)
    df_sorted = df.sort_values(by=columnas_fechas['inscripcion'], ascending=False)
    ultima_inscripcion = df_sorted.drop_duplicates(subset=id_col, keep='first')

    # 3. Número de altas previas al inicio del periodo
    altas_anteriores = df[df[columnas_fechas['inscripcion']] < fecha_corte_inicio] \
        .groupby(id_col).size().reset_index(name='NumAltasAntesDelPeriodo')

    # 4. Flag si tuvo alguna alta previa
    altas_flag = altas_anteriores.copy()
    altas_flag['TuvoAltasPrevias'] = True
    altas_flag = altas_flag[[id_col, 'TuvoAltasPrevias']]

    # 5. Última alta previa al inicio del periodo (fecha)
    ultima_alta_previa = df[df[columnas_fechas['inscripcion']] < fecha_corte_inicio] \
        .groupby(id_col)[columnas_fechas['inscripcion']].max().reset_index(name='UltimaAltaPrevia')

    # 6. Tiempo (meses) desde la última alta previa al inicio del periodo
    ultima_alta_previa['MesesDesdeUltimaAltaPrevia'] = (
        (fecha_corte_inicio - ultima_alta_previa['UltimaAltaPrevia']) / pd.Timedelta(days=30)
    ).astype(int)

    # --- Merge de todas las features ---
    df_final = ultima_inscripcion.copy()

    df_final = df_final.drop(columns=[columnas_fechas['antiguedad']]).merge(antiguedad_min, on=id_col, how='left')

    df_final = df_final.merge(altas_anteriores, on=id_col, how='left')
    df_final['NumAltasAntesDelPeriodo'] = df_final['NumAltasAntesDelPeriodo'].fillna(0).astype(int)

    df_final = df_final.merge(altas_flag, on=id_col, how='left')
    df_final['TuvoAltasPrevias'] = df_final['TuvoAltasPrevias'].fillna(False)

    df_final = df_final.merge(ultima_alta_previa[[id_col, 'MesesDesdeUltimaAltaPrevia']], on=id_col, how='left')
    df_final['MesesDesdeUltimaAltaPrevia'] = df_final['MesesDesdeUltimaAltaPrevia'].fillna(-1).astype(int)

    # 7. Duración del abono actual en meses hasta fecha de corte
    df_final['MesesDuracionAbonoActual'] = (
        (fecha_corte - df_final[columnas_fechas['inscripcion']]) / pd.Timedelta(days=30)
    ).astype(int)

    # Opcional: eliminar columnas que no quieras (por ejemplo 'FNacimiento')
    if 'FNacimiento' in df_final.columns:
        df_final = df_final.drop(columns='FNacimiento')

    return df_final


In [27]:
df_final_activos = crear_features_activos(df_abonos_limpios, fecha_corte_inicio='2024-09-01',  fecha_corte='2025-09-01')

In [28]:
df_final_activos

,IdPersona,Edad,FechaInscripcion,TipoAbonoActual,Sexo_Mujer,FAntiguedad,NumAltasAntesDelPeriodo,TuvoAltasPrevias,MesesDesdeUltimaAltaPrevia,MesesDuracionAbonoActual
0,117538,10,2025-09-01,AM00,True,2025-09-01,0,False,-1,0
1,109390,57,2025-09-01,MA00,True,2022-06-20,0,False,-1,0
2,116058,35,2025-09-01,AA00,True,2025-02-15,0,False,-1,0
3,96089,46,2025-09-01,AC03,False,2018-03-15,1,True,77,0
4,96101,49,2025-09-01,AA00,True,2025-09-01,0,False,-1,0
...,...,...,...,...,...,...,...,...,...,...
5322,10700,79,1994-06-14,AG00,False,1981-08-07,1,True,367,380
5323,18596,65,1992-05-01,CL08,False,1992-05-01,1,True,393,405
5324,8228,56,1979-07-05,CL13,True,1979-07-05,1,True,549,562
5325,849,85,1958-08-04,CL04,False,1958-08-04,1,True,804,816


In [29]:
eda_basica(df_final_activos, nombre_df="Clientes Final Activos")


📋 Análisis EDA básico de: Clientes Final Activos

📌 Tipos de Variables:
🔢 Variables Numéricas: ['IdPersona', 'Edad', 'NumAltasAntesDelPeriodo', 'MesesDesdeUltimaAltaPrevia', 'MesesDuracionAbonoActual']
🔠 Variables Categóricas: ['FechaInscripcion', 'TipoAbonoActual', 'Sexo_Mujer', 'FAntiguedad', 'TuvoAltasPrevias']

🕳️ Variables con valores nulos:
✅ No hay valores nulos en el dataset.

📎 Filas duplicadas:
✅ No hay filas duplicadas.

📎 Columnas duplicadas:
✅ No hay columnas duplicadas.


### - ARCHIVO CLIENTES ALTA DE 1/9/24 al 1/9/25

In [30]:
abonado_alta = load_dataset('../data/Altes abonats 01.09.2024 a 01.09.2025.xlsx')

In [32]:
columnas_a_eliminar = ['DireccionCompleta', 'FNacimiento']
columnas_a_renombrar = {'TipoAbono': 'TipoAbonoAlta', 'FAntiguedad':'FAntiguedadAlta'}
columnas_numericas = ['Edad']
columnas_fechas = ['FAntiguedadAlta', 'FAntiguedad']

altas = preparar_datos_iniciales(abonado_alta, columnas_a_eliminar, columnas_a_renombrar,
    columnas_numericas,   columnas_fechas)

In [34]:
eda_basica(altas, nombre_df="Clientes con Altas")


📋 Análisis EDA básico de: Clientes con Altas

📌 Tipos de Variables:
🔢 Variables Numéricas: ['IdPersona', 'Edad']
🔠 Variables Categóricas: ['TipoAbonoAlta', 'FAntiguedadAlta', 'FechaAlta', 'Sexo']

🕳️ Variables con valores nulos:
✅ No hay valores nulos en el dataset.

📎 Filas duplicadas:
✅ No hay filas duplicadas.

📎 Columnas duplicadas:
✅ No hay columnas duplicadas.


In [36]:
# Filtrar por 'FAntiguedadAlta'
altas = filtrar_por_fecha(altas, 'FAntiguedadAlta', '2025-09-01')

# También  filtramos por 'FechaAlta'
altas_filtrado = filtrar_por_fecha(altas, 'FechaAlta', '2025-09-01')
altas_filtrado

,IdPersona,TipoAbonoAlta,FAntiguedadAlta,FechaAlta,Edad,Sexo
0,3461,EMPF - FAMILIAR EMPLEADO,2024-09-25,2024-09-25,62,Mujer
1,13180,AG00 - GENT GRAN (MES DE 65 ANYS),2024-10-15,2024-10-15,78,Hombre
2,14339,AG00 - GENT GRAN (MES DE 65 ANYS),2025-09-01,2025-09-01,78,Mujer
3,14339,CR01 - CARNET ROSA (TARJETA REDUÏDA),2024-09-01,2024-09-01,78,Mujer
4,15072,AA0 - PROMO 9'90€ (PRIMER MES),2024-09-09,2024-09-09,84,Mujer
...,...,...,...,...,...,...
3590,117425,AA00 - ADULTS ( 26 A 64 ANYS ),2025-09-01,2025-09-01,30,Hombre
3591,117426,AA00 - ADULTS ( 26 A 64 ANYS ),2025-09-01,2025-09-01,46,Mujer
3592,117427,JO00 - QUOTA DESCOMPTE JOVE (DE 18 A 25 ANYS),2025-09-01,2025-09-01,20,Hombre
3593,117537,AM00 - INFANTIL (DE 6 A 17 ANYS),2025-09-01,2025-09-01,16,Hombre


In [37]:
tipos_a_excluir = ["EMP0 - EMPLEADOS CLUB SIN CUOTA",  "EMP1 - EMPLEATS D'ALTRES EMPRESES",  "CL02 - SOCIS NUMERARIS",    "CL01 - SOCIS D'HONOR", "AA0 - PROMO 9'90€ (PRIMER MES)"]

altas_filtrado = excluir_valores(altas, 'TipoAbonoAlta', tipos_a_excluir)

In [39]:
altas_filtrado_limpios = cambios_nombre_abonos( altas_filtrado,  columna='TipoAbonoAlta',   mapeo_manual=mapeo_manual_abonos)
altas_filtrado_limpios

,IdPersona,TipoAbonoAlta,FAntiguedadAlta,FechaAlta,Edad,Sexo
0,3461,EMPF,2024-09-25,25/09/2024,62,Mujer
1,13180,AG00,2024-10-15,15/10/2024,78,Hombre
2,14339,AG00,2025-09-01,01/09/2025,78,Mujer
3,14339,CR01,2024-09-01,01/09/2024,78,Mujer
4,15072,AR00,2024-09-09,09/09/2024,84,Mujer
...,...,...,...,...,...,...
3521,117425,AA00,2025-09-01,01/09/2025,30,Hombre
3522,117426,AA00,2025-09-01,01/09/2025,46,Mujer
3523,117427,JO00,2025-09-01,01/09/2025,20,Hombre
3524,117537,AM00,2025-09-01,01/09/2025,16,Hombre


In [47]:
def crear_features_altas_periodo(df: pd.DataFrame, fecha_inicio: str,  fecha_corte: str,
                                     fecha_col: str = 'FechaAlta',  id_col: str = 'IdPersona') -> pd.DataFrame:
    """
    Genera features a partir de altas filtradas por periodo temporal.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame que contiene las altas con columna de fecha.
    fecha_inicio : str
        Fecha inicial del periodo (formato 'YYYY-MM-DD').
    fecha_corte : str
        Fecha final del periodo (formato 'YYYY-MM-DD').
    fecha_col : str, opcional
        Nombre de la columna de fechas en df (por defecto 'FechaAlta').
    id_col : str, opcional
        Nombre columna identificadora de persona (por defecto 'IdPersona').

    Retorna:
    --------
    pd.DataFrame
        DataFrame agrupado por id_col con features:
        - NumAltasEnPeriodo: cantidad de altas en el periodo
        - FechaPrimeraAltaEnPeriodo: fecha de la primera alta en periodo
        - MesesDesdePrimeraAltaEnPeriodo: meses desde la primera alta al corte
    """
    df = df.copy()

    # Convertir columna de fecha a datetime, asumiendo día primero
    df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')

    # Convertir fechas de corte
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_corte = pd.to_datetime(fecha_corte)

    # Filtrar por periodo
    df_periodo = df[(df[fecha_col] >= fecha_inicio) & (df[fecha_col] <= fecha_corte)]

    # Agrupar para crear features
    altas_agg = df_periodo.groupby(id_col).agg(
        NumAltasEnPeriodo=(fecha_col, 'count'),
        FechaPrimeraAltaEnPeriodo=(fecha_col, 'min')
    ).reset_index()

    # Calcular meses desde la primera alta hasta fecha de corte
    altas_agg['MesesDesdePrimeraAltaEnPeriodo'] = (
        (fecha_corte - altas_agg['FechaPrimeraAltaEnPeriodo']) / pd.Timedelta(days=30)
    ).astype(int)

    return altas_agg

In [48]:
df_altas_features = crear_features_altas_periodo( altas_filtrado_limpios,  fecha_inicio='2024-09-01',  fecha_corte='2025-09-01',
                                                     fecha_col='FechaAlta',    id_col='IdPersona')
df_altas_features

,IdPersona,NumAltasEnPeriodo,FechaPrimeraAltaEnPeriodo,MesesDesdePrimeraAltaEnPeriodo
0,3461,1,2024-09-25,11
1,13180,1,2024-10-15,10
2,14339,2,2024-09-01,12
3,15072,1,2024-09-09,11
4,15471,1,2025-02-26,6
...,...,...,...,...
3385,117425,1,2025-09-01,0
3386,117426,1,2025-09-01,0
3387,117427,1,2025-09-01,0
3388,117537,1,2025-09-01,0


UNIÓN DATAFRAME DE ACTIVOS CON ALTAS

In [56]:
def preparar_union_activos_altas(df_activos: pd.DataFrame,  altas_agg: pd.DataFrame,  id_col: str = 'IdPersona',
                                     fecha_inscripcion_col: str = 'FechaInscripcion',    fecha_corte: str = '2025-09-01') -> pd.DataFrame:
    """
    Une el DataFrame de activos con las features de altas y prepara variables adicionales.

    Parámetros:
    -----------
    df_activos : pd.DataFrame
        DataFrame con los datos de los activos.
    altas_agg : pd.DataFrame
        DataFrame con las features agregadas de altas (por persona).
    id_col : str, opcional
        Nombre de la columna identificadora (por defecto 'IdPersona').
    fecha_inscripcion_col : str, opcional
        Nombre de la columna con la fecha de inscripción (por defecto 'FechaInscripcion').
    fecha_corte : str, opcional
        Fecha de corte para cálculos de duración (por defecto '2025-09-01').

    Retorna:
    --------
    pd.DataFrame
        DataFrame combinado y con nuevas columnas preparadas:
        - EsChurn: False por defecto (puedes usar para marcar bajas después)
        - NumAltasEnPeriodo: relleno 0 si no hay datos
        - MesesDesdePrimeraAltaEnPeriodo: relleno -1 si no hay datos
        - FechaFin: igual a fecha_corte
        - VidaGymMeses: meses entre fecha de inscripción y fecha fin
    """
    df = df_activos.copy()
    df_altas = altas_agg.copy()

    # Convertir fecha_corte a timestamp
    fecha_corte_ts = pd.to_datetime(fecha_corte)

    # Merge left para conservar todos los activos
    df_merged = df.merge(df_altas, on=id_col, how='left')

    # Crear columna 'EsChurn' a False (por defecto)
    df_merged['EsChurn'] = False

    # Rellenar NaNs en columnas de altas con valores por defecto
    if 'NumAltasEnPeriodo' in df_merged.columns:
        df_merged['NumAltasEnPeriodo'] = df_merged['NumAltasEnPeriodo'].fillna(0).astype(int)
    if 'MesesDesdePrimeraAltaEnPeriodo' in df_merged.columns:
        df_merged['MesesDesdePrimeraAltaEnPeriodo'] = df_merged['MesesDesdePrimeraAltaEnPeriodo'].fillna(-1).astype(int)
    if 'FechaPrimeraAltaEnPeriodo' in df_merged.columns:
        df_merged = df_merged.drop(columns=['FechaPrimeraAltaEnPeriodo'])

    # Asegurar que la fecha de inscripción es datetime
    df_merged[fecha_inscripcion_col] = pd.to_datetime(df_merged[fecha_inscripcion_col], errors='coerce')

    # Definir fecha fin = fecha_corte
    df_merged['FechaFin'] = fecha_corte_ts

    # Calcular VidaGym en días y luego en meses
    df_merged['VidaGymDias'] = (df_merged['FechaFin'] - df_merged[fecha_inscripcion_col]).dt.days
    df_merged['VidaGymMeses'] = df_merged['VidaGymDias'] / 30

    # Eliminar columna auxiliar días
    df_merged = df_merged.drop(columns=['VidaGymDias'])

    return df_merged


In [57]:
df_completo_activos_altas = preparar_union_activos_altas( df_activos=df_final_activos,altas_agg=df_altas_features, id_col='IdPersona',
                                                             fecha_inscripcion_col='FechaInscripcion',  fecha_corte='2025-09-01')
df_completo_activos_altas

,IdPersona,Edad,FechaInscripcion,TipoAbonoActual,Sexo_Mujer,FAntiguedad,NumAltasAntesDelPeriodo,TuvoAltasPrevias,MesesDesdeUltimaAltaPrevia,MesesDuracionAbonoActual,NumAltasEnPeriodo,MesesDesdePrimeraAltaEnPeriodo,EsChurn,FechaFin,VidaGymMeses
0,117538,10,2025-09-01,AM00,True,2025-09-01,0,False,-1,0,1,0,False,2025-09-01,0.000000
1,109390,57,2025-09-01,MA00,True,2022-06-20,0,False,-1,0,0,-1,False,2025-09-01,0.000000
2,116058,35,2025-09-01,AA00,True,2025-02-15,0,False,-1,0,1,6,False,2025-09-01,0.000000
3,96089,46,2025-09-01,AC03,False,2018-03-15,1,True,77,0,0,-1,False,2025-09-01,0.000000
4,96101,49,2025-09-01,AA00,True,2025-09-01,0,False,-1,0,1,0,False,2025-09-01,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5322,10700,79,1994-06-14,AG00,False,1981-08-07,1,True,367,380,0,-1,False,2025-09-01,380.066667
5323,18596,65,1992-05-01,CL08,False,1992-05-01,1,True,393,405,0,-1,False,2025-09-01,405.866667
5324,8228,56,1979-07-05,CL13,True,1979-07-05,1,True,549,562,0,-1,False,2025-09-01,562.000000
5325,849,85,1958-08-04,CL04,False,1958-08-04,1,True,804,816,0,-1,False,2025-09-01,816.666667


### - ARCHIVO CLIENTES DE BAJA DE 1/9/24 al 1/9/25

In [40]:
abonado_bajas = load_dataset('../data/Baixes Abonats 01.09.2024 a 01.09.2025.xlsx')
abonado_bajas = abonado_bajas[1:].reset_index(drop=True)

In [41]:
abonado_bajas

,IdPersona,TipoAbono,FAntiguedad,FechaAlta,FechaBaja,MotivoBaja,Sexo,Edad,DireccionCompleta,FNacimiento,GENERAL,Total
0,7801,AG00 - GENT GRAN (MES DE 65 ANYS),26/06/2024,26/06/2024,01/10/2024,Raons personals,Mujer,83,"CARRER CHAPI, 77",06/12/1941,1,1
1,10389,AR00 - TARJA ROSA (TARJETA GRATUÏTA),12/06/1981,01/01/2012,01/02/2025,Problemas de salud,Mujer,81,"PASSEIG UNIVERSAL, 56-58 2-2",14/01/1944,1,1
2,11780,AG00 - GENT GRAN (MES DE 65 ANYS),30/09/1982,01/03/2018,01/08/2025,Vacances,Mujer,80,"CARRER SALSES, 100 BLQ-C 2-2",05/03/1945,1,1
3,13103,AG00 - GENT GRAN (MES DE 65 ANYS),27/04/1984,17/03/2011,01/03/2025,No Utilitzar les Instal.lacions,Hombre,85,"CARRER MESTRE DALMAU, 22 1",25/03/1940,1,1
4,13104,AG00 - GENT GRAN (MES DE 65 ANYS),27/04/1984,17/03/2011,01/03/2025,No Utilitzar les Instal.lacions,Mujer,83,"CARRER MESTRE DALMAU, 22 1",13/08/1942,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3560,117348,AA00 - ADULTS ( 26 A 64 ANYS ),25/08/2025,25/08/2025,25/08/2025,(nulo),Hombre,36,"CARRER CAMPOAMOR, 24",25/05/1989,1,1
3561,117352,AA00 - ADULTS ( 26 A 64 ANYS ),26/08/2025,26/08/2025,28/08/2025,(nulo),Mujer,50,"CARRER RECTORIA, 27",28/02/1975,1,1
3562,117370,AG00 - GENT GRAN (MES DE 65 ANYS),29/08/2025,29/08/2025,29/08/2025,(nulo),Mujer,73,"CARRER CARTELLA, 134 3-2",27/03/1952,1,1
3563,117421,AA0 - PROMO 9'90€ (PRIMER MES),01/09/2025,01/09/2025,01/09/2025,(nulo),Mujer,26,"CARRER COIMBRA, 30 2-4",15/04/1999,1,1


In [42]:
columnas_a_eliminar = ['DireccionCompleta', 'MotivoBaja','FNacimiento' , 'GENERAL', 'Total']
columnas_a_renombrar = {'TipoAbono': 'TipoAbonoBaja', 'FAntiguedad':'FAntiguedadBaja', 'FechaAlta':'FechaAltaBaja'}
columnas_numericas = ['Edad']
columnas_fechas = ['FAntiguedadAlta', 'FAntiguedad']

bajas = preparar_datos_iniciales(abonado_bajas, columnas_a_eliminar, columnas_a_renombrar,
    columnas_numericas,   columnas_fechas)

In [43]:
eda_basica(bajas, nombre_df="Clientes con Bajas")


📋 Análisis EDA básico de: Clientes con Bajas

📌 Tipos de Variables:
🔢 Variables Numéricas: ['IdPersona', 'Edad']
🔠 Variables Categóricas: ['TipoAbonoBaja', 'FAntiguedadBaja', 'FechaAltaBaja', 'FechaBaja', 'Sexo']

🕳️ Variables con valores nulos:
✅ No hay valores nulos en el dataset.

📎 Filas duplicadas:
✅ No hay filas duplicadas.

📎 Columnas duplicadas:
✅ No hay columnas duplicadas.


#### - FEATURE ENGINEERING ARCHIVO BAJAS

Se realiza como en el archivo de abonados activos una pequeño filtro. Se excluyen los abonados que se han dado de baja que no forman parte y no interfieren directamente al numero de socios. Tal como los abonados con abonos antiguos, que no pagan, y también empleados del gimnasio.

In [44]:
tipos_a_excluir = ["EMP0 - EMPLEADOS CLUB SIN CUOTA",  "EMP1 - EMPLEATS D'ALTRES EMPRESES",  "CL02 - SOCIS NUMERARIS",    "CL01 - SOCIS D'HONOR", "AA0 - PROMO 9'90€ (PRIMER MES)"]

bajas_filtrado = excluir_valores(bajas, 'TipoAbonoBaja', tipos_a_excluir)

In [45]:
bajas_filtrado_limpios = cambios_nombre_abonos( bajas_filtrado,  columna='TipoAbonoBaja',   mapeo_manual=mapeo_manual_abonos)
bajas_filtrado_limpios

,IdPersona,TipoAbonoBaja,FAntiguedadBaja,FechaAltaBaja,FechaBaja,Sexo,Edad
0,7801,AG00,26/06/2024,26/06/2024,01/10/2024,Mujer,83
1,10389,AR00,12/06/1981,01/01/2012,01/02/2025,Mujer,81
2,11780,AG00,30/09/1982,01/03/2018,01/08/2025,Mujer,80
3,13103,AG00,27/04/1984,17/03/2011,01/03/2025,Hombre,85
4,13104,AG00,27/04/1984,17/03/2011,01/03/2025,Mujer,83
...,...,...,...,...,...,...,...
3459,117296,AC00,09/08/2025,09/08/2025,01/09/2025,Mujer,29
3460,117316,AF00,16/08/2025,16/08/2025,20/08/2025,Mujer,31
3461,117348,AA00,25/08/2025,25/08/2025,25/08/2025,Hombre,36
3462,117352,AA00,26/08/2025,26/08/2025,28/08/2025,Mujer,50


In [46]:
bajas_filtrado_limpios = codificar_one_hot(bajas_filtrado_limpios, 'Sexo')
bajas_filtrado_limpios

,IdPersona,TipoAbonoBaja,FAntiguedadBaja,FechaAltaBaja,FechaBaja,Edad,Sexo_Mujer
0,7801,AG00,26/06/2024,26/06/2024,01/10/2024,83,True
1,10389,AR00,12/06/1981,01/01/2012,01/02/2025,81,True
2,11780,AG00,30/09/1982,01/03/2018,01/08/2025,80,True
3,13103,AG00,27/04/1984,17/03/2011,01/03/2025,85,False
4,13104,AG00,27/04/1984,17/03/2011,01/03/2025,83,True
...,...,...,...,...,...,...,...
3459,117296,AC00,09/08/2025,09/08/2025,01/09/2025,29,True
3460,117316,AF00,16/08/2025,16/08/2025,20/08/2025,31,True
3461,117348,AA00,25/08/2025,25/08/2025,25/08/2025,36,False
3462,117352,AA00,26/08/2025,26/08/2025,28/08/2025,50,True


In [49]:
def crear_features_bajas_periodo(df: pd.DataFrame, fecha_inicio: str,  fecha_corte: str,
                                    fecha_col: str = 'FechaBaja',   id_col: str = 'IdPersona') -> pd.DataFrame:
    """
    Genera features a partir de bajas filtradas por periodo temporal.

    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame que contiene las bajas con columna de fecha.
    fecha_inicio : str
        Fecha inicial del periodo (formato 'YYYY-MM-DD').
    fecha_corte : str
        Fecha final del periodo (formato 'YYYY-MM-DD').
    fecha_col : str, opcional
        Nombre de la columna de fechas en df (por defecto 'FechaBaja').
    id_col : str, opcional
        Nombre columna identificadora de persona (por defecto 'IdPersona').

    Retorna:
    --------
    pd.DataFrame
        DataFrame agrupado por id_col con features:
        - NumBajasEnPeriodo: cantidad de bajas en el periodo
        - FechaUltimaBajaEnPeriodo: fecha de la última baja en periodo
        - MesesDesdeUltimaBaja: meses desde la última baja al corte
    """
    df = df.copy()

    # Convertir columna de fecha a datetime, asumiendo día primero
    df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')

    # Convertir fechas de corte
    fecha_inicio = pd.to_datetime(fecha_inicio)
    fecha_corte = pd.to_datetime(fecha_corte)

    # Filtrar por periodo
    df_periodo = df[(df[fecha_col] >= fecha_inicio) & (df[fecha_col] <= fecha_corte)]

    # Agrupar para crear features
    bajas_agg = df_periodo.groupby(id_col).agg(
        NumBajasEnPeriodo=(fecha_col, 'count'),
        FechaUltimaBajaEnPeriodo=(fecha_col, 'max')
    ).reset_index()

    # Calcular meses desde la última baja hasta fecha de corte
    bajas_agg['MesesDesdeUltimaBaja'] = (
        (fecha_corte - bajas_agg['FechaUltimaBajaEnPeriodo']) / pd.Timedelta(days=30)
    ).astype(int)

    return bajas_agg


In [50]:
df_bajas_features = crear_features_bajas_periodo( bajas_filtrado_limpios,  fecha_inicio='2024-09-01',  fecha_corte='2025-09-01',
                                                     fecha_col='FechaBaja',    id_col='IdPersona')
df_bajas_features

,IdPersona,NumBajasEnPeriodo,FechaUltimaBajaEnPeriodo,MesesDesdeUltimaBaja
0,7801,1,2024-10-01,11
1,10389,1,2025-02-01,7
2,11780,1,2025-08-01,1
3,13103,1,2025-03-01,6
4,13104,1,2025-03-01,6
...,...,...,...,...
3210,117296,1,2025-09-01,0
3211,117316,1,2025-08-20,0
3212,117348,1,2025-08-25,0
3213,117352,1,2025-08-28,0


In [51]:
def merge_bajas_info(df_bajas_features, bajas_filtrado_limpios, id_col='IdPersona',
                     cols_a_unir=['IdPersona', 'Edad', 'Sexo_Mujer', 'FAntiguedadBaja', 'FechaAltaBaja', 'TipoAbonoBaja', 'FechaBaja'],
                     fecha_orden_col='FechaAltaBaja'):
    """
    Une la información histórica de bajas con las features de bajas.

    Parámetros:
    - df_bajas_features: DataFrame con las features de bajas.
    - bajas_filtrado_limpios: DataFrame histórico filtrado y limpio de bajas.
    - id_col: columna identificadora para merge (por defecto 'IdPersona').
    - cols_a_unir: columnas que quieres unir desde bajas_filtrado_limpios.
    - fecha_orden_col: columna para ordenar para obtener la última baja (por defecto 'FechaAltaBaja').

    Retorna:
    - DataFrame resultante del merge.
    """
    # Ordenamos y obtenemos la última baja por IdPersona
    df_hist_abonados = bajas_filtrado_limpios.sort_values(fecha_orden_col).drop_duplicates(id_col, keep='last')

    # Merge con df_bajas_features
    bajas_completas = df_bajas_features.merge(
        df_hist_abonados[cols_a_unir],
        on=id_col,
        how='left'
    )

    return bajas_completas


In [52]:
bajas_completas = merge_bajas_info( df_bajas_features,  bajas_filtrado_limpios,   id_col='IdPersona',
    cols_a_unir=['IdPersona', 'Edad', 'Sexo_Mujer', 'FAntiguedadBaja', 'FechaAltaBaja', 'TipoAbonoBaja', 'FechaBaja'],
    fecha_orden_col='FechaAltaBaja'
)

In [53]:
bajas_completas

,IdPersona,NumBajasEnPeriodo,FechaUltimaBajaEnPeriodo,MesesDesdeUltimaBaja,Edad,Sexo_Mujer,FAntiguedadBaja,FechaAltaBaja,TipoAbonoBaja,FechaBaja
0,7801,1,2024-10-01,11,83,True,26/06/2024,26/06/2024,AG00,01/10/2024
1,10389,1,2025-02-01,7,81,True,12/06/1981,01/01/2012,AR00,01/02/2025
2,11780,1,2025-08-01,1,80,True,30/09/1982,01/03/2018,AG00,01/08/2025
3,13103,1,2025-03-01,6,85,False,27/04/1984,17/03/2011,AG00,01/03/2025
4,13104,1,2025-03-01,6,83,True,27/04/1984,17/03/2011,AG00,01/03/2025
...,...,...,...,...,...,...,...,...,...,...
3210,117296,1,2025-09-01,0,29,True,09/08/2025,09/08/2025,AC00,01/09/2025
3211,117316,1,2025-08-20,0,31,True,16/08/2025,16/08/2025,AF00,20/08/2025
3212,117348,1,2025-08-25,0,36,False,25/08/2025,25/08/2025,AA00,25/08/2025
3213,117352,1,2025-08-28,0,50,True,26/08/2025,26/08/2025,AA00,28/08/2025


In [54]:
def preparar_bajas(df_bajas,  col_antiguedad='FAntiguedadBaja',  col_fecha_baja='FechaBaja',   col_fecha_alta='FechaAltaBaja',  col_tipo_abono='TipoAbonoBaja'):
    """
    Limpia y prepara el DataFrame de bajas para análisis.

    Parámetros:
    - df_bajas: DataFrame original de bajas.
    - col_antiguedad: columna de antigüedad en bajas (fecha).
    - col_fecha_baja: columna de fecha de baja.
    - col_fecha_alta: columna de fecha de alta (para renombrar).
    - col_tipo_abono: columna de tipo abono (para renombrar).

    Retorna:
    - DataFrame modificado con fechas convertidas, cálculo de vida en meses, flag EsChurn, y columnas renombradas.
    """
    df = df_bajas.copy()

    # Convertir a datetime
    df[col_antiguedad] = pd.to_datetime(df[col_antiguedad], dayfirst=True)
    df[col_fecha_baja] = pd.to_datetime(df[col_fecha_baja], dayfirst=True)

    # Calcular vida en días y meses
    df['VidaGymDias'] = (df[col_fecha_baja] - df[col_antiguedad]).dt.days
    df['VidaGymMeses'] = df['VidaGymDias'] / 30
    df = df.drop(columns='VidaGymDias')

    # Flag de churn
    df['EsChurn'] = True

    # Renombrar columnas para homogeneizar
    df = df.rename(columns={
        col_antiguedad: 'FAntiguedad',
        col_fecha_alta: 'FechaInscripcion',
        col_tipo_abono: 'TipoAbonoActual',
        col_fecha_baja: 'FechaFin'
    })

    return df

In [55]:
df_bajas_preparadas = preparar_bajas(bajas_completas, col_antiguedad='FAntiguedadBaja',  col_fecha_baja='FechaBaja',
                                     col_fecha_alta='FechaAltaBaja', col_tipo_abono='TipoAbonoBaja')
df_bajas_preparadas

,IdPersona,NumBajasEnPeriodo,FechaUltimaBajaEnPeriodo,MesesDesdeUltimaBaja,Edad,Sexo_Mujer,FAntiguedad,FechaInscripcion,TipoAbonoActual,FechaFin,VidaGymMeses,EsChurn
0,7801,1,2024-10-01,11,83,True,2024-06-26,26/06/2024,AG00,2024-10-01,3.233333,True
1,10389,1,2025-02-01,7,81,True,1981-06-12,01/01/2012,AR00,2025-02-01,531.333333,True
2,11780,1,2025-08-01,1,80,True,1982-09-30,01/03/2018,AG00,2025-08-01,521.533333,True
3,13103,1,2025-03-01,6,85,False,1984-04-27,17/03/2011,AG00,2025-03-01,497.266667,True
4,13104,1,2025-03-01,6,83,True,1984-04-27,17/03/2011,AG00,2025-03-01,497.266667,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3210,117296,1,2025-09-01,0,29,True,2025-08-09,09/08/2025,AC00,2025-09-01,0.766667,True
3211,117316,1,2025-08-20,0,31,True,2025-08-16,16/08/2025,AF00,2025-08-20,0.133333,True
3212,117348,1,2025-08-25,0,36,False,2025-08-25,25/08/2025,AA00,2025-08-25,0.000000,True
3213,117352,1,2025-08-28,0,50,True,2025-08-26,26/08/2025,AA00,2025-08-28,0.066667,True


UNIÓN DATAFRAME DE ACTIVOS CON BAJAS Y GESTIÓN DE CONFLICTOS

In [58]:
def analizar_inconsistencias(df_bajas_preparadas, df_completo_activos_altas):
    """
    Analiza IDs que aparecen tanto en bajas como en activos y calcula porcentaje de casos erróneos.

    Parámetros:
    - df_bajas_preparadas: DataFrame con bajas preparadas.
    - df_completo_activos_altas: DataFrame con activos y altas combinadas.

    Retorna:
    - dict con IDs en ambos DataFrames y porcentajes respecto a ambos totales.
    """
    # IDs de bajas que también están en activos
    ids_en_ambos = df_bajas_preparadas.loc[
        df_bajas_preparadas['IdPersona'].isin(df_completo_activos_altas['IdPersona']),
        'IdPersona'
    ].unique()

    bajas_en_activos = df_bajas_preparadas[
        df_bajas_preparadas['IdPersona'].isin(df_completo_activos_altas['IdPersona'])
    ]

    # Casos erróneos = cantidad de IDs en ambos
    N_erroneos = len(ids_en_ambos)

    # Cálculo de porcentajes
    total_bajas = len(df_bajas_preparadas)
    porcentaje_erroneos_bajas = (N_erroneos / total_bajas) * 100 if total_bajas > 0 else 0

    total_activos = len(df_completo_activos_altas)
    porcentaje_erroneos_activos = (N_erroneos / total_activos) * 100 if total_activos > 0 else 0

    print(f"Los casos inconsistentes representan el {porcentaje_erroneos_bajas:.2f}% del total de bajas.")
    print(f"Los casos inconsistentes representan el {porcentaje_erroneos_activos:.2f}% del total de activos.")

    return {
        'ids_en_ambos': ids_en_ambos,
        'bajas_en_activos': bajas_en_activos,
        'porcentaje_erroneos_bajas': porcentaje_erroneos_bajas,
        'porcentaje_erroneos_activos': porcentaje_erroneos_activos
    }


In [59]:
resultados = analizar_inconsistencias(df_bajas_preparadas, df_completo_activos_altas)

Los casos inconsistentes representan el 11.07% del total de bajas.
Los casos inconsistentes representan el 6.68% del total de activos.


Durante el análisis de los datos, se detectaron 356 registros (aproximadamente el 11% del dataframe de bajas y el 7% del dataframe de activos) correspondientes a abonados que aparecen simultáneamente como activos y dados de baja en el mismo periodo. Esta situación es inconsistente con la lógica del negocio, dado que un abonado no debería estar activo y dado de baja a la vez.

Tras evaluar las posibles causas y la complejidad que implica resolver estas inconsistencias (por ejemplo, abonados que se dieron de baja y volvieron a darse de alta en cortos períodos), se decidió eliminar estos casos para:

Evitar introducir ruido y confusión en el modelo predictivo.

Simplificar el análisis y garantizar la coherencia de los datos.

Mantener la calidad y fiabilidad del conjunto de datos utilizados para entrenamiento y validación.

Esta decisión fue tomada con la consideración de que la proporción de registros eliminados es relativamente baja y no debería afectar significativamente el rendimiento ni la generalización del modelo.

In [60]:
def eliminar_inconsistencias(df_activos, df_bajas, ids_inconsistentes):
    """
    Elimina los registros de ambos DataFrames cuyos IdPersona estén en la lista de inconsistencias.

    Parámetros:
    - df_activos: DataFrame de activos (df_completo_activos_altas)
    - df_bajas: DataFrame de bajas (df_bajas_preparadas)
    - ids_inconsistentes: lista o array de IdPersona inconsistentes (ids_en_ambos)

    Retorna:
    - Tuple con los DataFrames limpios (activos_sin_incons, bajas_sin_incons)
    """
    activos_limpio = df_activos[~df_activos['IdPersona'].isin(ids_inconsistentes)].copy()
    activos_limpio= activos_limpio.reset_index(drop=True)
    bajas_limpio = df_bajas[~df_bajas['IdPersona'].isin(ids_inconsistentes)].copy()
    bajas_limpio= bajas_limpio.reset_index(drop=True)
    
    return activos_limpio, bajas_limpio

In [61]:
df_completo_activos_altas, df_bajas_preparadas = eliminar_inconsistencias(df_completo_activos_altas, df_bajas_preparadas, resultados['ids_en_ambos'])
df_bajas_preparadas

,IdPersona,NumBajasEnPeriodo,FechaUltimaBajaEnPeriodo,MesesDesdeUltimaBaja,Edad,Sexo_Mujer,FAntiguedad,FechaInscripcion,TipoAbonoActual,FechaFin,VidaGymMeses,EsChurn
0,10389,1,2025-02-01,7,81,True,1981-06-12,01/01/2012,AR00,2025-02-01,531.333333,True
1,13103,1,2025-03-01,6,85,False,1984-04-27,17/03/2011,AG00,2025-03-01,497.266667,True
2,13104,1,2025-03-01,6,83,True,1984-04-27,17/03/2011,AG00,2025-03-01,497.266667,True
3,15072,1,2025-07-01,2,84,True,2024-09-09,09/09/2024,AR00,2025-07-01,9.833333,True
4,15168,1,2024-11-01,10,38,True,2024-08-27,27/08/2024,JO00,2024-11-01,2.200000,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2854,117275,1,2025-09-01,0,51,False,2025-08-06,06/08/2025,AA00,2025-09-01,0.866667,True
2855,117276,1,2025-09-01,0,37,True,2025-08-06,06/08/2025,AA00,2025-09-01,0.866667,True
2856,117277,1,2025-09-01,0,7,False,2025-08-06,06/08/2025,AM00,2025-09-01,0.866667,True
2857,117282,1,2025-09-01,0,36,False,2025-08-07,07/08/2025,AA00,2025-09-01,0.833333,True


In [62]:
pd.set_option('display.max_columns', None)

In [63]:
def preparar_df_final(df_activos, df_bajas):
    """
    Une los DataFrames de activos y bajas, y prepara el DataFrame final con las conversiones
    de fechas, tratamiento de valores faltantes y ajustes en columnas booleanas.

    Parámetros:
    - df_activos: DataFrame con usuarios activos (preparados)
    - df_bajas: DataFrame con usuarios dados de baja (preparados)

    Retorna:
    - df_usuarios_final: DataFrame final listo para análisis
    """
    import pandas as pd

    # Concatenar activos y bajas
    df_usuarios_final = pd.concat([df_activos, df_bajas], ignore_index=True)

    # Convertir columnas de fechas a datetime (manejar errores y formato día primero)
    for col in ['FechaInscripcion', 'FAntiguedad', 'FechaFin', 'FechaUltimaBajaEnPeriodo']:
        if col in df_usuarios_final.columns:
            df_usuarios_final[col] = pd.to_datetime(df_usuarios_final[col], errors='coerce', dayfirst=True)

    # Para usuarios churn (EsChurn=True), ajustar 'TuvoAltasPrevias'
    df_usuarios_final.loc[df_usuarios_final['EsChurn'] == True, 'TuvoAltasPrevias'] = False
    df_usuarios_final['TuvoAltasPrevias'] = df_usuarios_final['TuvoAltasPrevias'].astype(bool)

    # Para usuarios activos (EsChurn=False), rellenar NaNs en columnas de bajas con 0
    cols_bajas = ['NumBajasEnPeriodo', 'MesesDesdeUltimaBaja']
    for col in cols_bajas:
        if col in df_usuarios_final.columns:
            df_usuarios_final.loc[df_usuarios_final['EsChurn'] == False, col] = \
                df_usuarios_final.loc[df_usuarios_final['EsChurn'] == False, col].fillna(0)

    # Para usuarios churn (EsChurn=True), rellenar NaNs en columnas de altas con -1
    cols_altas = ['NumAltasAntesDelPeriodo', 'TuvoAltasPrevias', 'MesesDesdeUltimaAltaPrevia',
                  'MesesDuracionAbonoActual', 'NumAltasEnPeriodo', 'MesesDesdePrimeraAltaEnPeriodo']
    for col in cols_altas:
        if col in df_usuarios_final.columns:
            df_usuarios_final.loc[df_usuarios_final['EsChurn'] == True, col] = \
                df_usuarios_final.loc[df_usuarios_final['EsChurn'] == True, col].fillna(-1)
    
    return df_usuarios_final

In [64]:
df_usuarios_final = preparar_df_final(df_completo_activos_altas, df_bajas_preparadas)

In [65]:
df_usuarios_final

,IdPersona,Edad,FechaInscripcion,TipoAbonoActual,Sexo_Mujer,FAntiguedad,NumAltasAntesDelPeriodo,TuvoAltasPrevias,MesesDesdeUltimaAltaPrevia,MesesDuracionAbonoActual,NumAltasEnPeriodo,MesesDesdePrimeraAltaEnPeriodo,EsChurn,FechaFin,VidaGymMeses,NumBajasEnPeriodo,FechaUltimaBajaEnPeriodo,MesesDesdeUltimaBaja
0,117538,10,2025-09-01,AM00,True,2025-09-01,0.0,False,-1.0,0.0,1.0,0.0,False,2025-09-01,0.000000,0.0,NaT,0.0
1,109390,57,2025-09-01,MA00,True,2022-06-20,0.0,False,-1.0,0.0,0.0,-1.0,False,2025-09-01,0.000000,0.0,NaT,0.0
2,116058,35,2025-09-01,AA00,True,2025-02-15,0.0,False,-1.0,0.0,1.0,6.0,False,2025-09-01,0.000000,0.0,NaT,0.0
3,96089,46,2025-09-01,AC03,False,2018-03-15,1.0,True,77.0,0.0,0.0,-1.0,False,2025-09-01,0.000000,0.0,NaT,0.0
4,96101,49,2025-09-01,AA00,True,2025-09-01,0.0,False,-1.0,0.0,1.0,0.0,False,2025-09-01,0.000000,0.0,NaT,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7825,117275,51,2025-08-06,AA00,False,2025-08-06,-1.0,False,-1.0,-1.0,-1.0,-1.0,True,2025-09-01,0.866667,1.0,2025-09-01,0.0
7826,117276,37,2025-08-06,AA00,True,2025-08-06,-1.0,False,-1.0,-1.0,-1.0,-1.0,True,2025-09-01,0.866667,1.0,2025-09-01,0.0
7827,117277,7,2025-08-06,AM00,False,2025-08-06,-1.0,False,-1.0,-1.0,-1.0,-1.0,True,2025-09-01,0.866667,1.0,2025-09-01,0.0
7828,117282,36,2025-08-07,AA00,False,2025-08-07,-1.0,False,-1.0,-1.0,-1.0,-1.0,True,2025-09-01,0.833333,1.0,2025-09-01,0.0


#### COMPROBACIÓN FINAL DEL DATASET

In [68]:
df_usuarios_final[df_usuarios_final['IdPersona']==96089	]

,IdPersona,Edad,FechaInscripcion,TipoAbonoActual,Sexo_Mujer,FAntiguedad,NumAltasAntesDelPeriodo,TuvoAltasPrevias,MesesDesdeUltimaAltaPrevia,MesesDuracionAbonoActual,NumAltasEnPeriodo,MesesDesdePrimeraAltaEnPeriodo,EsChurn,FechaFin,VidaGymMeses,NumBajasEnPeriodo,FechaUltimaBajaEnPeriodo,MesesDesdeUltimaBaja
3,96089,46,2025-09-01,AC03,False,2018-03-15,1.0,True,77.0,0.0,0.0,-1.0,False,2025-09-01,0.0,0.0,NaT,0.0


In [69]:
df_usuarios_final['EsChurn'].value_counts()

EsChurn
False    4971
True     2859
Name: count, dtype: int64

In [67]:
df_usuarios_final['TipoAbonoActual'].value_counts()

TipoAbonoActual
AA00     2590
JO00      958
FA00      903
AG00      683
MA00      501
AM00      434
AR00      317
QM01      282
AF00      222
AC00      203
CR01      167
AT01      155
TEMP       78
FM01       74
AT00       49
AA03       44
AVET       23
VIP        18
AP00       18
AA12       17
AP03       14
EMPF       10
AG03        9
MA03        8
FA12        7
AC03        6
JO03        5
AF03        5
NI00        4
AG12        3
AR03        3
CL08        2
AM03        2
CR03        2
AR12        1
AA06        1
AC12        1
MA12        1
MA06        1
APG03       1
T14         1
QM03        1
T15         1
T12         1
T07         1
CL05        1
CL13        1
CL04        1
Name: count, dtype: int64

In [70]:
df_usuarios_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7830 entries, 0 to 7829
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   IdPersona                       7830 non-null   int64         
 1   Edad                            7830 non-null   int64         
 2   FechaInscripcion                7830 non-null   datetime64[ns]
 3   TipoAbonoActual                 7830 non-null   object        
 4   Sexo_Mujer                      7830 non-null   bool          
 5   FAntiguedad                     7830 non-null   datetime64[ns]
 6   NumAltasAntesDelPeriodo         7830 non-null   float64       
 7   TuvoAltasPrevias                7830 non-null   bool          
 8   MesesDesdeUltimaAltaPrevia      7830 non-null   float64       
 9   MesesDuracionAbonoActual        7830 non-null   float64       
 10  NumAltasEnPeriodo               7830 non-null   float64       
 11  Mese

# GUARDAMOS DATAFRAME RESULTANTE 

Ese dataframe se guarda para reutilizarse posteriormente para crear el archivo final para el modelo. Se guarda en formato CSV.

Comentario: Faltan criterios de preparación del dataframe para el modelo, se hace posteriormente en el notebook de archivo final

In [71]:
# Guardar el DataFrame en un archivo CSV
df_usuarios_final.to_csv('../data/abonados_final_pre_modelo.csv', index=False)